# [CPSC 310](https://github.com/GonzagaCPSC310) Data Mining
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)

# Apriori
What are our learning objectives for this lesson?
* Introduce the apriori algorithm

Content used in this lesson is based upon information in the following sources:
* Dr. Shawn Bowers' Data Mining notes

## "Market Basket" Analysis
In Market Basket Analysis
* Dataset consists of "transactions" (one per "row")
* Each transaction has a set of items (an "itemset")

Grocery store example ...
* T1 {bread, cheese, milk}
* T2 {bread, cheese, fish, milk, sugar}
* T3 {bread, cheese, milk}

Note: T1 and T3 have same itemset

* Often itemsets ordered to make calculations easier (example later)
* Implication is items in itemset "go" together (e.g., purchased in a sale)

## Association Rules for Itemsets
If left itemset THEN right itemset
* E.g., IF {bread, sugar} THEN {cheese, milk}
* Implies buying bread and sugar is associated with buying cheese and milk
* Again, causality is not implied

We'll write the above as $L \rightarrow R$ (e.g., ${bread, sugar} \rightarrow {cheese, milk}$)

## Support for Itemsets
For an itemset S
* $support(S) = \frac{count(S)}{N}$
    * where $N$ = total # of transactions
    * The $count(S)$ is the number of transactions $T$ where $S \subseteq T$

If $S = L \cup R$ for $Rule_1$: IF $L$ THEN $R$
* $support(S)$ (or similarly, $support(Rule_1)$) is same as before $N_{both}/N_{total}$
* Just within subset or equal condition ($\subseteq$)

## Confidence for Itemsets
If $S = L \cup R$ for a rule $Rule_1$: IF $L$ THEN $R$
* $confidence(Rule_1) = \frac{count(L \cup R)}{count(L)} = \frac{count(S)}{count(L)}$
    * Same as before: $N_{both}/N_{left}$

Often, only interested in "confident" and "supported" rules, i.e., where the confidence is at least $minconf$ (e.g., 80%) and support is at least $minsup$ (e.g., 1%)

### Lab Task 1
Calculate support and confidence for some rules for the following Example Market Basket Analysis (Fake) data:

|transaction |itemset|
|-|-|
|1 |{b, c, m}|
|2 |{b, c, e, m, s}|
|3 |{b}|
|4 |{c, e, s}|
|5 |{c}|
|6 |{b, c, s}|
|7 |{c, e, s}|
|8 |{c, e}|

Where:
* b = bread
* c = cheese
* e = eggs
* m = milk
* s = sugar


## The Apriori Algorithm
Leverages the fact that:
* If an itemset is supported, all of its (non-empty) subsets are also supported
* Removing an item cannot reduce # of matching transactions
* e.g., {c,m} $\rightarrow$ {b} is supported, and so {c} $\rightarrow$ {b} is also supported

The fact implies the following
* Let $L_k$ be the set of supported itemsets with $k$ items
* If $L_k$ is empty (not supported), then $L_{k+1}, L_{k+2}, ...$ will be empty too (i.e., these sets will not be supported either either)
* ... cant improve support by adding items

Basic Apriori Algorithm ("bottom up")
1. Find $L_1$ (the set of 1-item supported itemsets)
2. Generate $L_2$ from $L_1$
3. Continue ($L_3$ from $L_2$, $L_4$ from $L_3$, etc.) until $L_k = \emptyset$
4. Generate rules from resulting itemsets

Generating $L_k$ from $L_{k−1}$ (Step 2)
* Create a "candidate" itemset $C_k$ from $L_{k−1}$ ... assumes sorted itemsets
    * (i) For each $A \in L_{k−1}$ and $B \in L_{k−1} (A \neq B)$
        * (ii)     If $A[0:-1] == B[0:-1]$
            * (iii)        Add $A \cup B$ to $C_k$ **unless**
                * (iv)             a $k - 1$ subset of $A \cup B \notin L_{k−1}$
* Set $L_k$ to supported itemsets in $C_k$

Step (ii) implies all but last item in $A$ and $B$ match (for $A$ and $B$ sorted)
Step (iv) prunes search space
* If a $k - 1$ subset is not in $L_{k-1}$ then $A \cup B$ must not be supported
* This is the essential fact exploited by Apriori
* That is, adding an item won't increase support

Supported itemsets are $L_{2} \cup ... \cup L_k$ where $k$ is the last non-empty set
* These are also represented solely by $L_k$ considering each elements subsets

### Lab Task 2
Find all supported itemsets assuming $minsup$ = 25%
* $L_1$ = {b}, {c}, {e}, {m}, {s}
* $C_2$ = {b, c}, {b, e}, {b, m}, {b, s}, {c, e}, {c, m}, {c, s}, {e, m}, {e, s}, {m, s}
* $L_2$ = {b, c}, {b, m}, {b, s}, {c, e}, {c, m}, {c, s}, {e, s}
* $C_3$ = {b, c, m}, {b, c, s}, {c, e, s}
* $L_3$ = {b, c, m}, {b, c, s}, {c, e, s}
* $C_4$ = $\emptyset$
* $L_4$ = $\emptyset$

### Lab Task 3
Write a function to generate all k-1 subsets of a list of size k

```python
def k_1_subsets(itemset):
    result = []
    for i in range(len(itemset)):
        result.append(itemset[:i] + itemset[i+1:]
    return result
```
-- or --
```python
def k_1_subsets(itemset):
    n = len(itemset)
    return [itemset[:i]+itemset[i+1:] for i in range(n)]
```


### Generating Rules from Itemsets
Naive (brute force) algorithm
* Given itemset $S$, find all RHSs
    * Start with 1-item RHS, move to 2-item RHS, etc.
    * For each rule, remaining items not in RHS become LHS

### Lab Task 3
Find all rules for $S = {b, c, m}$ and compute confidence

|LHS |RHS |Confidence ($N_{both}/N_{left}$)|
|-|-|-|
|{c, m}$ \rightarrow$  |{b} |100% (2/2)|
|{b, m}$ \rightarrow$ |{c} |100% (2/2)|
|{b, c}$ \rightarrow$ |{m} |66% (2/3)|
|{m}$ \rightarrow$ |{b, c} |100% (2/2)|
|{c}$ \rightarrow$ |{b, m} |29% (2/7)|
|{b}$ \rightarrow$ |{c, m} |50% (2/4)|

* We find confidence for each
* And keep rules satisfying $minconf$ (e.g., at 80%)

The Problem with this approach:
* For an itemset with $k$ items
* There are $2^{k}$ - 2 RHSs! (again, all subsets)
    * "-2" since we don't count set of all items and empty set (for RHS)
* Thus, exponential in the size of itemsets

Instead use the following fact:
* Moving items from LHS to RHS cannot increase rule confidence
    * If $R1 = A \cup B \rightarrow C$
    * We have $conf(R1) = \frac{count(A \cup B \cup C)}{count(A \cup B)}$
    * If $R2 = A \rightarrow B \cup C$ ... move $B$ to RHS
    * We have $conf(R2) = \frac{count(A \cup B \cup C)}{count(A)}$
    * And always true that $count(A) \geq count(A \cup B)$
        * Again, adding more items cannot increase # of matching transactions

From the example:
* {b, c} $\rightarrow$ {m}... $conf$ = 66% (2/3)
* {c} $\rightarrow$ {b, m}... $conf$ = 29% (2/7)
* {b} $\rightarrow$ {c, m}... $conf$ = 50% (2/4)

* Given the first rule, we knew second two would be $\leq$ 66%
* Which is below, e.g., $minconf$ = 80%

This means we can "shortcircuit" rule generation
* If no rules with a RHS of size $k$ are "confident"... stop searching for rules
    * Since algorithm proceeds $k = 1, k = 2, ...$
    * Confident means confidence $\geq minconf$
* Can also record RHSs not to consider
    * e.g., any that are a superset of $\{m\}$
    
## Apriori Rule Interestingness Measures
* There can still be many rules satisfying $minconf$ and $minsup$
* Various metrics for further determining "interestingness"

### Leverage
* The difference between support of $A \cup B$ and support of $A$ and $B$ if independent
$$leverage(A \rightarrow B) = support(A \cup B) - support(A) \times support(B)$$
* Interested in "improvement" of support
    * e.g., if $support(A)$ = 10% and $support(B)$ = 10%, if independent we'd expect them to occur together approximately 1% of the time
* Typically set $minleverage$ low, e.g., 0.0001 (improve 1 in every 10,000 transactions)

### Lift
* Similar to leverage
* But instead of difference, measures how many more times $A$ and $B$ occur together (than if independent)
$$lift(A \rightarrow B) = \frac{support(A \cup B)}{support(A) \times support(B)}$$
* Lift values greater than 1 are considered "interesting"
* But with all metrics, you must use common sense!

Q: Can we apply Apriori to tabular data?
* YES! (assuming categorical or discretized values)
* Each row becomes a transaction (itemset) of attribute-value pair items
* Itemsets restricted though, e.g., $\{s = 1, j = 3\}$ but not $\{s = 1, s = 2\}$ for attributes $s$ and $j$.
* This affects how itemsets are combined in Apriori